# 4 - PDF RAG OpenAI y Langchain


<br>
<br>

<img src="https://raw.githubusercontent.com/Hack-io-AI/ai_images/main/rag_2.webp" style="width:400px;"/>

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#1---OpenAI-API-KEY" data-toc-modified-id="1---OpenAI-API-KEY-1">1 - OpenAI API KEY</a></span></li><li><span><a href="#2---Probando-GPT4-desde-LangChain" data-toc-modified-id="2---Probando-GPT4-desde-LangChain-2">2 - Probando GPT4 desde LangChain</a></span></li><li><span><a href="#3---Cargando-archivo-PDF" data-toc-modified-id="3---Cargando-archivo-PDF-3">3 - Cargando archivo PDF</a></span></li><li><span><a href="#4---Chunks" data-toc-modified-id="4---Chunks-4">4 - Chunks</a></span></li><li><span><a href="#5---Modelo-de-Embedding" data-toc-modified-id="5---Modelo-de-Embedding-5">5 - Modelo de Embedding</a></span></li><li><span><a href="#6---Guardado-en-ChromaDB" data-toc-modified-id="6---Guardado-en-ChromaDB-6">6 - Guardado en ChromaDB</a></span></li><li><span><a href="#7---Carga-desde-Chroma" data-toc-modified-id="7---Carga-desde-Chroma-7">7 - Carga desde Chroma</a></span></li><li><span><a href="#8---Prompt-template" data-toc-modified-id="8---Prompt-template-8">8 - Prompt template</a></span></li><li><span><a href="#9---Cadena" data-toc-modified-id="9---Cadena-9">9 - Cadena</a></span></li><li><span><a href="#11---Más-preguntas" data-toc-modified-id="11---Más-preguntas-10">11 - Más preguntas</a></span></li></ul></div>

## 1 - OpenAI API KEY

Para llevar a cabo este proyecto, necesitaremos una API KEY de OpenAI para utilizar el modelo GPT-4 Turbo. Esta API KEY se puede obtener en https://platform.openai.com/api-keys. Solo se muestra una vez, por lo que debe guardarse en el momento en que se obtiene. Por supuesto, necesitaremos crear una cuenta para obtenerla.

Guardamos la API KEY en un archivo .env para cargarla con la biblioteca dotenv y usarla como una variable de entorno. Este archivo se agrega a .gitignore para asegurar que no pueda verse si subimos el código a GitHub, por ejemplo.

In [ ]:
# importamos la API KEY

import os                           # libreria del sistema operativo
from dotenv import load_dotenv      # carga variables de entorno 


load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

## 2 - Probando GPT4 desde LangChain

Vamos a probar la conexión de LangChain con el modelo GPT-4. Simplemente preguntaremos quién es el CEO de Apple.

In [ ]:
from langchain_openai.chat_models import ChatOpenAI   

modelo = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model='gpt-4-turbo')

respuesta = modelo.invoke("Who is Apple's CEO?")

respuesta.content

## 3 - Cargando archivo PDF 

Ahora, cargamos el archivo PDF [Formulario 10-K de Apple de 2023](https://s2.q4cdn.com/470004039/files/doc_earnings/2023/q4/filing/_10-K-Q4-2023-As-Filed.pdf) previamente descargado. Un 10-K es un informe completo que una empresa que cotiza en bolsa presenta anualmente sobre su desempeño financiero y que es requerido por la Comisión de Bolsa y Valores de EE. UU. (SEC). Este informe contiene mucho más detalle que el informe anual de una empresa, que se envía a sus accionistas antes de la reunión anual para elegir a los directores de la empresa.

In [ ]:
os.listdir('../pdfs')

In [ ]:
from langchain_community.document_loaders import PyPDFDirectoryLoader

In [ ]:
# loads PDF file page by page

loader = PyPDFDirectoryLoader('../pdfs/')

paginas = loader.load()

In [ ]:
len(paginas)

In [ ]:
# primera pagina

paginas[0]  

## 4 - Chunks

El `PyPDFDirectoryLoader` utiliza una instancia de TextSplitter, específicamente el `RecursiveCharacterTextSplitter` por defecto, para manejar la división de documentos. Este enfoque ayuda a descomponer archivos PDF grandes o colecciones de archivos en fragmentos manejables para su posterior procesamiento. El cargador garantiza que cada fragmento sea manejable y conserve la metadatos necesarias, como números de página, que son importantes para hacer referencias y mantener la integridad de los documentos fuente durante el procesamiento.


In [ ]:
chunks = loader.load_and_split()

In [ ]:
len(chunks)

In [ ]:
chunks[55]

## 5 - Modelo de Embedding

Los embeddings transforman los datos, especialmente los datos textuales, en un formato, generalmente un vector de números, que los algoritmos de aprendizaje automático pueden procesar de manera efectiva. Estos embeddings capturan las relaciones contextuales y los significados semánticos de palabras, frases o documentos, lo que permite diversas aplicaciones en IA.

In [ ]:
from langchain_openai.embeddings import OpenAIEmbeddings


vectorizador = OpenAIEmbeddings()

## 6 - Guardado en ChromaDB

Chroma DB es una base de datos de vectores de código abierto diseñada para almacenar y recuperar embeddings de vectores de manera eficiente. Es especialmente útil para mejorar los LLMs al proporcionar contexto relevante a las consultas de los usuarios. Chroma DB permite almacenar embeddings junto con metadatos, que luego pueden ser utilizados por LLMs o para motores de búsqueda semántica sobre datos textuales.

Ahora, almacenamos los fragmentos en la base de datos de vectores.

In [ ]:
from langchain_chroma import Chroma

chroma_db = Chroma.from_documents(chunks, vectorizador, persist_directory='../chroma_db')

## 7 - Carga desde Chroma 

Una vez que los datos están guardados, podemos realizar una búsqueda de los documentos más relevantes según nuestra consulta. Podemos buscar directamente mediante búsqueda por similitud, basada en la similitud de coseno, o podemos instanciar un objeto de recuperación para usar más adelante.

Por defecto, el tipo de búsqueda realizada por el recuperador, `search_type`, es por similitud y devuelve los resultados más relevantes según esa similitud. También podemos utilizar la similitud con un umbral para recuperar documentos que superen un cierto nivel de similitud con nuestra consulta. El recuperador también cuenta con un algoritmo llamado MMR (relevancia marginal máxima). El algoritmo de relevancia marginal máxima selecciona documentos en función de una combinación de cuáles son los más similares a las consultas, optimizando también para la diversidad. Lo hace encontrando ejemplos con embeddings que tienen la mayor similitud de coseno con las entradas y luego los agrega de manera iterativa, aplicando una penalización para aquellos que estén demasiado cerca de los ejemplos ya seleccionados.

Utilizaremos el algoritmo MMR para devolver 15 documentos. El parámetro `lambda_mult` se refiere a la diversidad de los resultados devueltos por MMR, siendo 1 la mínima diversidad y 0 la máxima. El valor predeterminado es 0.5. Pediremos un poco más de diversidad en su respuesta.


In [ ]:
consulta = 'What can you tell me about foreign exchange contracts?'

chroma_db = Chroma(persist_directory='../chroma_db', embedding_function=vectorizador)

docs = chroma_db.similarity_search(consulta, k=10)

len(docs)

In [ ]:
docs[5]

In [ ]:
recuperador = chroma_db.as_retriever(search_type='mmr', search_kwargs={'k': 15, 'lambda_mult': 0.25})

In [ ]:
recuperador

## 8 - Prompt template

Los prompt templates son recetas predefinidas para generar instrucciones para modelos de lenguaje.

Un template puede incluir instrucciones, contexto y preguntas específicas adecuadas para una tarea determinada. LangChain proporciona herramientas para crear y trabajar con templates de instrucciones y también busca crear templates independientes del modelo, para facilitar la reutilización de templates existentes en diferentes modelos de lenguaje.

In [ ]:
from langchain.prompts import ChatPromptTemplate

In [ ]:
template = '''
            Answer the question based on the context below. If you can't 
            answer the question, reply "I don't know".

            Context: {context}

            Question: {question}
            '''


prompt = ChatPromptTemplate.from_template(template)

## 9 - Cadena

Una "cadena" se refiere a una secuencia de componentes o pasos que están vinculados entre sí para realizar una tarea específica o un conjunto de tareas relacionadas con las operaciones de IA o LLMs. LangChain es una biblioteca diseñada para facilitar la creación y despliegue de aplicaciones de lenguaje, encadenando diferentes componentes, como modelos, bases de datos y lógica personalizada. Cada componente en la cadena maneja una parte específica de la tarea, y la salida de un componente sirve como entrada para el siguiente, creando un flujo de trabajo continuo que aprovecha tanto metodologías de IA como de software tradicional. Una cadena actúa efectivamente como una tubería, donde los datos fluyen a través de cada componente en la cadena, siendo transformados, mejorados o utilizados en cada paso.

En LangChain, el `StrOutputParser` analiza la salida del modelo directamente en un formato de cadena. Utilizaremos este parser al crear la secuencia en LangChain; será un enlace adicional en la cadena, permitiéndonos obtener directamente la respuesta del LLM en formato de cadena.

`RunnablePassthrough` permite que las entradas pasen sin cambios.

In [ ]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [ ]:
from langchain_core.runnables import RunnablePassthrough

In [ ]:
consulta

In [ ]:
cadena = {'context': recuperador, 'question': RunnablePassthrough()} | prompt | modelo | parser


respuesta = cadena.invoke(consulta)


respuesta

## 11 - Más preguntas

In [ ]:
consulta = 'What can you tell me about foreign exchange contracts?'

cadena.invoke(consulta)

In [ ]:
consulta = 'What are the main products?'

cadena.invoke(consulta)

In [ ]:
consulta = 'What can you tell me about legal proceedings?'

cadena.invoke(consulta)

In [ ]:
consulta = '''
        How the company’s business and reputation 
        are impacted by information technology system failures and network disruptions?
        '''


cadena.invoke(consulta)